In [8]:
%cd nanoMoD

/teamspace/studios/this_studio/nanoMoD


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [21]:
import torch
import wandb
from hydra import compose, initialize

from nanomod import utils
from nanomod.model import DnasSearchModel, GPT
from nanomod.dataset import get_dataloaders
from nanomod.configuration import set_config_store

In [11]:
set_config_store()
with initialize(version_base=None, config_path="config"):
    cfg = compose(config_name="config_search")

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/async_helpers.py:129: UserWarning: 
'config_search' is validated against ConfigStore schema with the same name.
This behavior is deprecated in Hydra 1.1 and will be removed in Hydra 1.2.
See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/automatic_schema_matching for migration instructions.
  coro.send(None)


In [15]:
wandb.init()
state_dict, model_config = utils.load_checkpoint()
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: eduardopacheco. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-ckpt:latest, 344.11MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.8


In [16]:
base_model = GPT(model_config)
base_model.load_state_dict(state_dict)

number of parameters: 29.96M


<All keys matched successfully>

In [26]:
model = DnasSearchModel(base_model, cfg.dnas)
model.freeze()

In [17]:
train_loader, val_loader = get_dataloaders(cfg.data)

In [18]:
inputs, targets = next(iter(train_loader))

In [22]:
with torch.no_grad():
    logits, loss = base_model(inputs, targets)

In [27]:
with torch.no_grad():
    logits, loss = model(inputs, targets)

In [28]:
loss

tensor(12.4444)

In [23]:
loss

tensor(5.2774)